In [1]:
#!/usr/bin/evn python3
# coding=utf-8

# 使用该魔法，不用写plt.show()
%matplotlib inline  

import warnings
# 忽略警告提示
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns



# 导入训练数据
data = pd.read_csv('./data/train.csv')
# 年龄缺失值处理
data['Age']=data['Age'].fillna(data['Age'].median())     # 用中位数填充
# data['Age']=data['Age'].fillna( data['Age'].meann())    # 用平均数填充

# 再次查看数据完整性
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [2]:
print(data['Sex'].unique())
data.loc[data['Sex'] == 'male','Sex'] = 0
data.loc[data['Sex'] == 'female','Sex'] = 1

['male' 'female']


In [3]:
print(data['Embarked'].unique())
data['Embarked'] = data['Embarked'].fillna('S')
data.loc[data['Embarked'] == 'S','Embarked'] = 0
data.loc[data['Embarked'] == 'C','Embarked'] = 1
data.loc[data['Embarked'] == 'Q','Embarked'] = 2

['S' 'C' 'Q' nan]


In [4]:
#线性回归预测
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold

# KFold 用于交叉验证
predictors = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
alg = LinearRegression() # 线性回归
kf = KFold(data.shape[0],n_folds=3,random_state=1) # 交叉验证

predications = []
for train,test in kf:
  train_predictors = (data[predictors].iloc[train,:]) # 将训练数据拿出来，取predictors的特征值，注意，有些无关紧要的值并没有出现在这里
  train_target=data["Survived"].iloc[train]  # 真实值
  alg.fit(train_predictors,train_target)  # 将线性回归应用到数据上，此时，有特征数据，有train_target值
  test_predictions = alg.predict(data[predictors].iloc[test,:])  # 预测并获取到测试结果
  predications.append(test_predictions)  # 将测试结果添加到列表中
# 线性回归获得到的值是0-1区间的值

import numpy as np

# The predictions are in three separate numpy arrays. Concatenate them into one.
# We concatenate them on axis 0,as they only have one axis.
predications = np.concatenate(predications,axis=0)
predications[predications>.5] == 1
predications[predications<=.5] == 0
accuracy = sum(predications[predications == data["Survived"]])/len(predications) # 准确率
print(accuracy)

#线性回归预测的结果大约为0.7833....

0.0


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# 逻辑回归预测
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

alg = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(alg,data[predictors],data["Survived"],cv=3)
print(scores.mean())

0.7878787878787877


In [8]:
#随机森林预测
from sklearn import cross_validation
from sklearn.emsemble import RandomForestClassifier

predictors = ["Age","Pclass","Sex","SibSp","Parch","Fare","Embarked"]

# Initialize our algorithm with the default parameters
# n_estimators is the number of trees we want to make
# min_samples_split is the minimum number of rows we need to make a split 样本的个数
# min_samples_leaf is the minimum number of samples we can have at the place where a tree branch ends # 叶子节点的个数
alg = RandomForestClassifier(random_state=1,n_estimators=10,min_samples_split=2,min_samples_leaf =1)
kf = cross_validation.KFold(data.shape[0],n_folds=3,random_state=1)
scores = cross_validation.cross_val_score(alg,data[predictors],data["Survived"],cv=kf)
print(scores.mean())
#预测结果：0.785634.....


ModuleNotFoundError: No module named 'sklearn.emsemble'

In [ ]:
# 改变决策树的参数：
alg = RandomForestClassifier(random_state=1,n_estimators=50,min_samples_split=4,min_samples_leaf =2)
kf = cross_validation.KFold(data.shape[0],n_folds=3,random_state=1)
scores = cross_validation.cross_val_score(alg,data[predictors],data["Survived"],cv=kf)
print(scores.mean())
#预测结果：0.8159......